In [10]:
import re

In [11]:

def check_text(text):
    # 判断是否包含全大写的英文
    if re.search(r'\b[A-Z]+\b', text):
        return True
    
    # 判断是否包含驼峰命名法的英文
    if re.search(r'\b[a-z][a-zA-Z0-9]*[a-z]\b', text):
        return True

    # 判断是否包含以http开头的网址
    if re.search(r'http://', text):
        return True

    # 判断是否包含E:\\或D:\\
    if re.search(r'[ED]:\\', text):
        return True

    # 判断是否包含{数字}、&、%、[{{文字}}]
    if re.search(r'\{\d+\}|&|%|\[\{\{.+\}\}\]|【#.*#】', text):
        return True

    # 如果都不包含以上条件，返回False
    return False

In [12]:
import pandas as pd

In [24]:
# 读取Excel文件
excel_file_path = '新加坡海外数据中心初始化多语数据-2023-10-08-13-52+西班牙语.xlsx'
df = pd.read_excel(excel_file_path)
count_normal = 0
# 创建空的DataFrame来保存符合条件的数据
result_df = pd.DataFrame(columns=['资源编码', '简体中文(源)', '参考语言(英文)'])

for index, row in df.iterrows():
    f_value = row['资源编码']
    g_value = str(row['简体中文(源)'])
    i_value = row['参考语言(英文)']
    if check_text(g_value):
        new_data = {'资源编码':[f_value],'简体中文(源)': [g_value], '参考语言(英文)': [i_value]}
        new_df = pd.DataFrame(new_data)
        # Concatenate the DataFrames
        result_df = pd.concat([result_df, new_df], ignore_index=True)
    else:
        if count_normal < 25181:
            new_data = {'资源编码':[f_value],'简体中文(源)': [g_value], '参考语言(英文)': [i_value]}
            new_df = pd.DataFrame(new_data)
            # Concatenate the DataFrames
            result_df = pd.concat([result_df, new_df], ignore_index=True)
        count_normal += 1
    if index % 100 == 0:
        print("The progress:", index)
result_csv_path = 'result.csv'
result_df.to_csv(result_csv_path, index=False)

print(f"符合条件的数据已保存到 {result_csv_path}")

The progress: 0
The progress: 100
The progress: 200
The progress: 300
The progress: 400
The progress: 500
The progress: 600
The progress: 700
The progress: 800
The progress: 900
The progress: 1000
The progress: 1100
The progress: 1200
The progress: 1300
The progress: 1400
The progress: 1500
The progress: 1600
The progress: 1700
The progress: 1800
The progress: 1900
The progress: 2000
The progress: 2100
The progress: 2200
The progress: 2300
The progress: 2400
The progress: 2500
The progress: 2600
The progress: 2700
The progress: 2800
The progress: 2900
The progress: 3000
The progress: 3100
The progress: 3200
The progress: 3300
The progress: 3400
The progress: 3500
The progress: 3600
The progress: 3700
The progress: 3800
The progress: 3900
The progress: 4000
The progress: 4100
The progress: 4200
The progress: 4300
The progress: 4400
The progress: 4500
The progress: 4600
The progress: 4700
The progress: 4800
The progress: 4900
The progress: 5000
The progress: 5100
The progress: 5200
The p

In [25]:
result_df.shape

(80000, 3)

In [14]:
result_df.head()

,简体中文(源),参考语言(英文),资源编码
0,请输入目标位置的横坐标 X 值,Please enter the X-axis value of the target po...,000142
1,请输入目标位置的纵坐标 Y 值,Please enter the Y-axis value of the target po...,000143
2,POST,POST,000166
3,请输入程序的名称或路径（例如notepad，calc）,Please enter the name or path of program (such...,000192
4,输入要停止的进程名（Calc，notepad）,"Enter name of the process to be stopped (calc,...",000198


In [17]:
result_df.dtypes

简体中文(源)     object
参考语言(英文)    object
资源编码        object
dtype: object

In [19]:
from datasets import load_dataset, DatasetDict, Dataset

In [31]:
dataset = Dataset.from_pandas(result_df)

In [45]:
dataset

Dataset({
    features: ['资源编码', '简体中文(源)', '参考语言(英文)'],
    num_rows: 80000
})

In [49]:
# Split into train and test/validation
train_testvalid = dataset.train_test_split(test_size=0.2, shuffle=True)
# Split test/validation into validation and test
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, shuffle=True)

# Create DatasetDict
data_dict = DatasetDict({
    "train": train_testvalid['train'],
    "validation": test_valid['train'],
    "test": test_valid['test'],
})

In [50]:
data_dict

DatasetDict({
    train: Dataset({
        features: ['资源编码', '简体中文(源)', '参考语言(英文)'],
        num_rows: 64000
    })
    validation: Dataset({
        features: ['资源编码', '简体中文(源)', '参考语言(英文)'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['资源编码', '简体中文(源)', '参考语言(英文)'],
        num_rows: 8000
    })
})

In [51]:
!pip install huggingface_hub

In [53]:
!huggingface-cli login

^C
